In [0]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip
!unzip drugsCom_raw.zip
!rm *.zip
!ls

--2019-11-27 15:52:53--  https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42989872 (41M) [application/x-httpd-php]
Saving to: ‘drugsCom_raw.zip’

drugsCom_raw.zip    100%[===================>]  41.00M  14.4MB/s    in 2.9s    

2019-11-27 15:53:02 (14.4 MB/s) - ‘drugsCom_raw.zip’ saved [42989872/42989872]

Archive:  drugsCom_raw.zip
  inflating: drugsComTest_raw.tsv    
  inflating: drugsComTrain_raw.tsv   
drugsComTest_raw.tsv  drugsComTrain_raw.tsv  sample_data


In [0]:
!wget nlp.stanford.edu/data/glove.6B.zip

--2019-11-27 15:53:20--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-11-27 15:53:20--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2019-11-27 15:53:21--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [0]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [0]:
!mkdir data

In [0]:
import numpy as np
import pandas as pd
import keras
from keras.preprocessing.text import Tokenizer,  text_to_word_sequence
from keras.layers import Embedding, Input, Dense, LSTM,Bidirectional
from keras import backend as K
from keras.models import Model
import nltk
from gensim.models import KeyedVectors
nltk.download('stopwords')
from sklearn.metrics import accuracy_score, f1_score
import re
from keras import regularizers
import tensorflow as tf
from nltk import tokenize
from nltk.corpus import stopwords
from keras.callbacks import EarlyStopping,ModelCheckpoint
from random import shuffle
from keras.models import load_model
%tensorflow_version 1.x

Using TensorFlow backend.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
MAX_FEATURES=200000
MAX_SENT_LEN=100
EMBED_SIZE=100
NUM_EPOCHS=25
REG_PARAM = 1e-13
OUTPUT_DIM = 3

In [0]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(32,32,32),
                 n_classes=3, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim)
        # Initialization
        X = np.empty((self.batch_size, *self.dim))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = np.load('data/out' + str(ID) + '.npy').reshape(MAX_SENT_LEN,)

            # Store class
            y[i] = self.labels[ID]

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

In [0]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [0]:
def embed_matrix(word_index,EMBED_DIR='glove.6B.100d.txt'):
    print("Loading embeddings from ",EMBED_DIR)
    embeddings_index = {}
    f = open(EMBED_DIR,'r')
    for line in f:
        values = line.split()
        coefs = np.asarray(values[-EMBED_SIZE:], dtype='float32')
        word = ''.join(values[:-EMBED_SIZE])
        embeddings_index[word] = coefs
    f.close()
    print('Total %s word vectors.' % len(embeddings_index))
    embedding_matrix = np.zeros((len(word_index) + 1, EMBED_SIZE))
    absent_words = 0
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
        else:
            absent_words += 1
    print('Total absent words are', absent_words, 'which is', "%0.2f" % (absent_words * 100 / len(word_index)), '% of total words')
    return embedding_matrix


In [0]:
def load_data(mode='train'):
    if mode=='train':
        df = pd.read_csv('drugsComTrain_raw.tsv',sep='\t')
        to_remove = np.random.choice(df[df['rating']>=7].index,size=92510,replace=False)
        df = df.drop(to_remove).reset_index()
        to_remove = np.random.choice(df[df['rating']<=4].index,size=25719,replace=False)
        df = df.drop(to_remove).reset_index()
        if OUTPUT_DIM==3:
          df = df.replace(to_replace={1:'negative',2:'negative',3:'negative',4:'negative',5:'neutral',6:'neutral',7:'positive',8:'positive',9:'positive',10:'positive'})
          df = df.replace(to_replace={'positive':2,'neutral':1,'negative':0})
        return pd.DataFrame(df['review']),df['rating'].to_numpy()
    if mode=='test':
        df = pd.read_csv('drugsComTest_raw.tsv', sep='\t')
        if OUTPUT_DIM==3:
          df = df.replace(to_replace={1:'negative',2:'negative',3:'negative',4:'negative',5:'neutral',6:'neutral',7:'positive',8:'positive',9:'positive',10:'positive'})
          df = df.replace(to_replace={'positive':2,'neutral':1,'negative':0})
        return pd.DataFrame(df['review']), pd.get_dummies(df['rating']).to_numpy()

In [0]:
def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    st_w = ['ourselves', 'hers', 'between', 'yourself', 'again', 'there', 'about', 'once', 'during', 'out',
            'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for','its', 'yours', 'such', 'into', 'of', 
            'itself', 'other', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 
            'below',  'we', 'these', 'your', 'his', 'through', 'me', 'were', 'her',  'himself', 
            'this', 'down',  'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'she', 'all', 
             'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on',  
            'yourselves', 'then', 'that', 'what', 'over', 'why', 'so', 'now', 'under', 
            'he', 'you', 'herself', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 
            'few', 'whom', 'being', 'if', 'theirs', 'my',  'a', 'by', 'doing', 'it', 'how',  'here', 'than']
    try:
        string = re.sub(r"\\", "", string)    
        string = re.sub(r"\'", "", string)    
        string = re.sub(r"\"", "", string) 
        string = re.sub(r"&#039;", "'",string)
        string = re.sub(r"\r","",string)
        string = re.sub(r"\n","",string)
        string = string.split()
        string = ' '.join([w for w in string if w not in st_w])
    except:
        print(string)
    return string.strip().lower()

In [0]:
texts = []
train,labels = load_data('train')
for idx in range(train['review'].shape[0]):
    text = clean_str(train['review'][idx])
    texts.append(text)

In [0]:
tokenizer = Tokenizer(num_words=MAX_FEATURES, oov_token=True)
tokenizer.fit_on_texts(texts)

In [0]:
data = np.zeros((len(texts), MAX_SENT_LEN), dtype='int32')
for i, sent in enumerate(texts):
    wordTokens = text_to_word_sequence(sent)
    k=0
    for _, word in enumerate(wordTokens):
        try:
          if k<MAX_SENT_LEN:
              data[i,k] = tokenizer.word_index[word]
              k=k+1
        except:
              print(word)
              pass
for i in range(len(data)):
  np.save("data/out"+str(i),data[i])              

In [0]:
partition = {}
idx = np.arange(len(data))
np.random.shuffle(idx)
train_samples = int(0.8*(len(data)))
partition['train']=idx[:train_samples]
partition['validation'] = idx[train_samples:]
label_dict = {}
for i in range(len(data)):
  label_dict[i] = labels[i]

In [0]:
params = {'dim': (MAX_SENT_LEN,),
          'batch_size': 500,
          'n_classes': OUTPUT_DIM,
          'shuffle': True}

In [0]:
training_generator = DataGenerator(partition['train'], label_dict, **params)
validation_generator = DataGenerator(partition['validation'], label_dict, **params)

In [0]:
l2_reg = regularizers.l2(REG_PARAM)
word_index= tokenizer.word_index
embedding_matrix = embed_matrix(word_index)
embedding_layer = Embedding(len(word_index) + 1,EMBED_SIZE,weights=[embedding_matrix], input_length=MAX_SENT_LEN, trainable=False)
word_input = Input(shape=(MAX_SENT_LEN,), dtype='float32')
word_sequences = embedding_layer(word_input)
word_lstm = Bidirectional(LSTM(150, return_sequences=False, kernel_regularizer=l2_reg))(word_sequences)
preds = Dense(OUTPUT_DIM,activation='softmax')(word_lstm)

Loading embeddings from  glove.6B.100d.txt
Total 400000 word vectors.
Total absent words are 9957 which is 31.73 % of total words











In [0]:
model = Model(word_input,preds)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc',f1])
earlystop = EarlyStopping(monitor='val_loss',min_delta=0,patience=10,verbose=0,restore_best_weights=True)
checkpoint = ModelCheckpoint('textrnn.h5', verbose=0, monitor='val_loss',save_best_only=True, mode='auto') 
history = model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    epochs =NUM_EPOCHS,callbacks=[checkpoint,earlystop])




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/25
68/68 [==============================] - 34s 503ms/step - loss: 1.0671 - acc: 0.4204 - f1: 0.0935 - val_loss: 1.0359 - val_acc: 0.4612 - val_f1: 0.1597
Epoch 2/25
68/68 [==============================] - 30s 448ms/step - loss: 1.0009 - acc: 0.4973 - f1: 0.3087 - val_loss: 0.9947 - val_acc: 0.4956 - val_f1: 0.3520
Epoch 3/25
68/68 [==============================] - 31s 453ms/step - loss: 0.9554 - acc: 0.5309 - f1: 0.4040 - val_loss: 0.9542 - val_acc: 0.5288 - val_f1: 0.4166
Epoch 4/25
68/68 [==============================] - 31s 454ms/step - loss: 0.9250 - acc: 0.5520 - f1: 0.4522 - val_loss: 0.9442 - val_acc: 0.5234 - val_f1: 0.4400
Epoch 5/25
68/68 [==============================] - 31s 453ms/step - loss: 0.8978 - acc: 0.5714 - f1: 0.4885 - val_loss: 0.9020 - val_acc: 0.5747 - val_f1: 0.4901
Epoch 6/25
68/68 [==============================] - 31s 458ms/step - loss: 0.8813 - acc

In [0]:
test, test_labels = load_data('test')

In [0]:
test_texts=[]
for idx in range(test['review'].shape[0]):
    text = clean_str(test['review'][idx])
    test_texts.append(text)

test_data = np.zeros((len(test_texts), MAX_SENT_LEN), dtype='int32')
for i, sent in enumerate(test_texts):
    wordTokens = text_to_word_sequence(sent)
    k=0
    for _, word in enumerate(wordTokens):
        try:
            if k<MAX_SENT_LEN:
                test_data[i,k] = tokenizer.word_index[word]
                k=k+1
        except:
            # print(word)
            pass

In [0]:
pred = model.predict(test_data,batch_size=1000)

In [0]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report,roc_auc_score
pred_labels = np.argmax(pred,axis=1)
act_labels = np.argmax(test_labels,axis=1)
print('Acc: ',accuracy_score(act_labels,pred_labels))
print('F1: ',f1_score(act_labels,pred_labels,average='weighted'))
print(confusion_matrix(act_labels,pred_labels))
print(classification_report(act_labels,pred_labels))


Acc:  0.6157608897816464
F1:  0.6672794246341747
[[ 8040  4033  1424]
 [ 1087  2830   912]
 [ 3392  9811 22237]]
              precision    recall  f1-score   support

           0       0.64      0.60      0.62     13497
           1       0.17      0.59      0.26      4829
           2       0.90      0.63      0.74     35440

    accuracy                           0.62     53766
   macro avg       0.57      0.60      0.54     53766
weighted avg       0.77      0.62      0.67     53766

